# Configure visualization with CSV files

## Motivation

As an alternative to loading data from AnnData-Zarr format, CSV files can be used. However, the CSV format is less efficient than Zarr because it does not allow for compression or chunking (splitting a large file into multiple smaller ones). However it is fine to use for very small datasets or for educational purposes (as CSV may be a more familiar format and therefore easier to debug).

In the below import statement, note the usage of the `CsvWrapper` class (e.g., instead of `AnnDataWrapper` used in the previous tutorial notebooks).

In [3]:
from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    FileType as ft,
    DataType as dt,
    CsvWrapper,
)
from os.path import join

In [4]:
vc = VitessceConfig(schema_version="1.0.15", name='Habib et al', description='COVID-19 Healthy Donor Brain')

## Add data via two CSV files

In the following code, we create one dataset, and run `add_object` on that dataset twice (note the two chained `add_object` calls). This is because the cell type annotations and UMAP coordinates are stored in two separate CSV files.

For more information about the file contents and how to define the `options` parameter, see the documentation at http://vitessce.io/docs/data-file-types/ for the file types with a `csv` suffix (i.e., `{data_type}.csv`).

In [5]:
dataset = vc.add_dataset(name='Brain').add_object(CsvWrapper(
        data_type=dt.OBS_SETS.value,
        csv_path=join("raw_data", "habib17.cell_type_annotations.csv"),
        options={
            "obsIndex": "index",
            "obsSets": [
                {
                    "name": "Cell Type",
                    "column": "CellType"
                }
            ]
        },
        coordination_values={
            "obsType": "cell"
        }
    )
).add_object(CsvWrapper(
        data_type=dt.OBS_EMBEDDING.value,
        csv_path=join("raw_data", "habib17.umap.csv"),
        options={
            "obsIndex": "index",
            "obsEmbedding": ["UMAP_1", "UMAP_2"]
        },
        coordination_values={
            "obsType": "cell",
            "embeddingType": "UMAP"
        }
    )
)

## Add views

In [6]:
scatterplot = vc.add_view(vt.SCATTERPLOT, dataset=dataset, mapping="UMAP")
cell_sets = vc.add_view(vt.OBS_SETS, dataset=dataset)
cell_set_sizes = vc.add_view(vt.OBS_SET_SIZES, dataset=dataset)

## Arrange views

In [7]:
vc.layout(scatterplot | (cell_sets / cell_set_sizes));

## Render the widget

In [8]:
vw = vc.widget()
vw

VitessceWidget(config={'version': '1.0.15', 'name': 'Habib et al', 'description': 'COVID-19 Healthy Donor Brai…